In [1]:
import numpy as np
import CHONK_XL as chxl
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import xsimlab as xs
import CHONK_cpp as ch
import zarr
import helplotlib as hpl
import helper


import ipymaps
import ipywidgets


%matplotlib widget
%load_ext xsimlab.ipython

In [2]:


@xs.process
class CustomParameters:
    label_array = xs.variable(intent = 'out', dims = (('y','x'), ('node')))
    label_list = xs.any_object()
    CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
    nx = xs.foreign(chxl.ChonkBase, "nx")
    ny = xs.foreign(chxl.ChonkBase, "ny")
    dx = xs.foreign(chxl.ChonkBase, "dx")
    dy = xs.foreign(chxl.ChonkBase, "dy")
    
    active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
#     landscape = xs.any_object()

    def initialize(self):

        self.label_list = []
    
        self.label_array = np.zeros((nx*ny), dtype = np.int)
        
        # Labelling the second half of the landscape as label 1
        self.label_array[round((nx*ny)/2):] = 1

        self.label_list.append(ch.label(0))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 1e-5;
        self.label_list[-1].Ks_modifyer = 1.2;
        self.label_list[-1].Kr_modifyer = 0.8;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.5;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.label_list.append(ch.label(1))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 1e-5;
        self.label_list[-1].Ks_modifyer = 1;
        self.label_list[-1].Kr_modifyer = 0.3;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.1;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.CHONK.initialise_label_list(self.label_list)
        self.CHONK.update_label_array(self.label_array.ravel())
        
        
@xs.process
class UpliftLandscape(chxl.Uplift):
	uplift_done = xs.variable(intent = "out")
	runner_done = xs.foreign(chxl.Runner, "runner_done")
	uplift = xs.variable(intent = 'out', dims = [('y','x'), ('node')])
	switch_time = xs.variable(intent = 'in')
	CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
	active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
	nx = xs.foreign(chxl.ChonkBase, "nx")
	ny = xs.foreign(chxl.ChonkBase, "ny")
	dx = xs.foreign(chxl.ChonkBase, "dx") 
	dy = xs.foreign(chxl.ChonkBase, "dy")
    
	rate = xs.variable()
	minrate = xs.variable(default=0)
	idx_N1 = xs.variable()
	idx_N2 = xs.variable()
	idx_N3 = xs.variable()
	idx_N4 = xs.variable()
    

	def initialize(self):
		self.uplift = np.zeros((ny,nx))
		self.uplift[self.idx_N1:self.idx_N2,:] = 1
		self.uplift[self.idx_N3:self.idx_N4,:] = 1
		self.uplift[self.idx_N2:self.idx_N3,:] = self.minrate
		from scipy.ndimage import gaussian_filter
		self.uplift = gaussian_filter(self.uplift, sigma=2)
		self.uplift /= self.uplift.max()
		self.uplift *= self.rate
		self.uplift[[-1,0],:] = 0
		self.done = False
        

	@xs.runtime(args=['step_delta','step_end'])
	def run_step(self, dt, timing):
		self.CHONK.add_external_to_surface_elevation_tp1(self.uplift.ravel() * dt)
# 		print(self.CHONK.get_neighbours_for_debugging(99,50))
		self.uplift_done = True
		
# landscape.uplift4StSt

In [3]:
model = xs.Model({"ChonkBase": chxl.ChonkBase,
                "Runner": chxl.Runner,
                "Topography": chxl.Topography,
#                 "Uplift": UpliftLandscapeStSt,
                "Uplift": UpliftLandscape,
#                 "Uplift": UpliftLandscapeNF1,
#                 "Uplift": chxl.Uplift,
                "Lake": chxl.Lake,
                "Precipitation": chxl.Precipitation,
#                 "DefaultParameters": chxl.DefaultParameters,
                "DefaultParameters": CustomParameters,
                "Flow": chxl.Flow,
                "Fluvial": chxl.Fluvial,
                "Hillslope": chxl.Hillslope
            })



In [4]:
ny,nx = 200,200
dy,dx = 100,100

In [5]:
# %create_setup model
import xsimlab as xs
time = np.arange(0,1e7,1000)
otime = time[::20]

# init_z = np.load("./initial_topo_100_100.npy")
init_z = np.random.rand(ny,nx)
rate = 0.5e-3
idx_N1 = 10
idx_N2 = 30
idx_N3 = 70
idx_N4 = 80
# init_z[0,:] = 1000
# init_z[0,:] = 0
U = np.zeros((ny,nx)) + 2e-4

Utime = 1e8
ds_in = xs.create_setup(
    model=model,
    clocks={
        'time': time,
        'otime': otime
    },
    master_clock='time',
    input_vars={
        'ChonkBase__dx': dx,
        'ChonkBase__dy': dy,
        'ChonkBase__nx': nx,
        'ChonkBase__ny': ny,
        'ChonkBase__depths_res_sed_proportions': 10,
        'ChonkBase__n_depth_sed_tracking': 50,
        'ChonkBase__boundary_conditions': "periodic_EW",
        'ChonkBase__strati_recorder': True,
        'ChonkBase__path_strati': "./strati_test",
        'ChonkBase__pref_strati': "test1",
        'ChonkBase__create_folder_strati': True,
        'Topography__initial_elevation': init_z,
        'Uplift__rate': rate,
        'Uplift__idx_N1': idx_N1,
        'Uplift__idx_N2': idx_N2,
        'Uplift__idx_N3': idx_N3,
        'Uplift__idx_N4': idx_N4,
        'Uplift__switch_time': Utime,
#         'Lake__method': ('batch', ['implicit', 'explicit']),
        'Lake__method': 'explicit',
        'Lake__Aincision_threshold': False,
        'Lake__evaporation': False,
        'Lake__evaporation_rate':  1.4,#('batch', np.arange(1,1.6,0.1)),
        'Flow__threshold_single_flow': 1e32,
        'Precipitation__precipitation_rate': 1,#('batch', np.arange(0.3,2,0.2)), #1
    },
    output_vars={
        'Topography__topography': 'otime',
        'Topography__sed_height': 'otime',
        'Flow__Qw': 'otime',
        'Flow__water_balance_checker': 'time',
        'Lake__lake_depth': 'otime',
        'Lake__topolake': 'otime',
        'Fluvial__Qs': 'otime',
        'Hillslope__Qs': 'otime',
        'ChonkBase__Q_sout_lab_N': 'otime',
        'ChonkBase__Q_sout_lab_S': 'otime',
        'ChonkBase__strati': 'otime',
    
    }
)


In [6]:
from dask.distributed import Client
if('client' in locals()):
    if(isinstance(client,Client)):
        client.close()
client = Client(n_workers=6, threads_per_worker=1)  


zg = zarr.group("rift_lake_explicit_NoLakeLuca.zarr", overwrite=True)
with model,xs.monitoring.ProgressBar():
    out_ds = ds_in.xsimlab.run(store = zg, parallel=False, scheduler=client)
#     out_ds = ds_in.xsimlab.run(store = zg)
#     out_ds = mod1.xsimlab.run()  
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0
# out_ds["topolake"] = out_ds["Topography__topography"] + out_ds["Lake__lake_depth"]

distributed.diskutils - INFO - Found stale lock file and directory '/Users/gailleto/Desktop/codes/CHONK/paper_example/dask-worker-space/worker-v7juk5n2', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/gailleto/Desktop/codes/CHONK/paper_example/dask-worker-space/worker-e7o2w07o', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/gailleto/Desktop/codes/CHONK/paper_example/dask-worker-space/worker-q442sg_b', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/gailleto/Desktop/codes/CHONK/paper_example/dask-worker-space/worker-u39wy668', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/gailleto/Desktop/codes/CHONK/paper_example/dask-worker-space/worker-_7r5as9i', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/gailleto/Desktop/codes/CHONK/paper_example/dask-worker-space/worker-ee148ewt', purging


             0% | initialize 

/var/folders/t0/7zf23f454vn3_kkgrbf4c1500000gp/T/ipykernel_2863/1523435931.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.label_array = np.zeros((nx*ny), dtype = np.int)


In [3]:
from ipyfastscape import TopoViz3d



# app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Topography__topography" )
app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Lake__topolake" )

app.show()

Output(layout=Layout(height='640px'))

In [9]:
#np.save("initial_topo_100_100.npy", out_ds.Topography__topography.values[-1])
# np.save("sed4testHS_CHONK.npy", out_ds.Topography__sed_height.sel({'otime':1.001e6}, method="nearest").values)
# np.save("sed4testtopo_CHONK.npy", out_ds.Topography__topography.sel({'otime':1.001e6}, method="nearest").values)

In [10]:
fig,ax = plt.subplots()
ax.plot(out_ds.time.values, out_ds.sel({'batch':1}).Flow__water_balance_checker.values)
# ax.plot(out_ds.time.values, np.abs(out_ds.Flow__water_balance_checker.values))
ax.set_yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'batch is not a valid dimension or coordinate'

In [ ]:
fig, ax = plt.subplots()
cb = ax.imshow(np.log10(out_ds["Flow__Qw"].sel({'batch':1, 'otime':9900000}).values), cmap = "Blues")
plt.colorbar(cb)

In [10]:
import matplotlib.gridspec as gridspec
import xarray as xr
import matplotlib
%matplotlib widget

font = {'family' : 'Helvetica Neue',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

out_ds = xr.open_zarr("rift_lake_evaporation_2.zarr/")
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0
desev = out_ds.Lake__evaporation_rate.values
# for ind,datev in zip(out_ds.batch.values, desev):
for ind,datev in zip([0],[0]):
#     tout_ds = out_ds.sel({"batch":ind})
    tout_ds = xr.open_zarr("rift_lake_implicit_vs_explicit_2.zarr/").sel({'batch':1})

    zumzumN = tout_ds.ChonkBase__Q_sout_lab_N.sum(dim = ['x'])
    zumzumS = tout_ds.ChonkBase__Q_sout_lab_S.sum(dim = ['x'])

    A = zumzumN.sel({'n_labels':0})
    B = A + zumzumN.sel({'n_labels':1})
    C = zumzumS.sel({'n_labels':0})
    D = C + zumzumS.sel({'n_labels':1})

    plt.close('all')


    fig =  plt.figure(constrained_layout=False, figsize = (12,8))

    grid = gridspec.GridSpec(100, 1, wspace=0, hspace=0)

    ax1,ax2 = fig.add_subplot(grid[0:50, 0]),fig.add_subplot(grid[50:, 0])

    ax1.fill_between(tout_ds.otime, 0,A, lw = 0, color = '#bdbdbd', label = "From the Southern half")
    ax1.fill_between(tout_ds.otime, A, B, lw = 0, color = '#474747', label = "From the Northern Half")

    ax2.fill_between(tout_ds.otime, 0,C, lw = 0, color = '#bdbdbd', label = "S label 0")
    ax2.fill_between(tout_ds.otime, C,D, lw = 0, color = '#474747', label = "S label 1")

    ax1.legend(loc = 'upper left')

    whylim = [0,140000]
    whylim2 = [0,140000]
    ax2.set_ylim(whylim2)
    ax1.set_ylim(whylim)
    ax2.invert_yaxis()

    ax1.get_yaxis().set_label_coords(-0.1,0.55)
    ax2.get_yaxis().set_label_coords(-0.1,0.7)

    ax1.grid(ls = '--', color = 'k', lw = 1, alpha = 0.6) 
    ax2.grid(ls = '--', color = 'k', lw = 1, alpha = 0.6) 

    ax2.set_xlabel("Time (yrs)")
    ax2.set_ylabel(r"$Q_s$ [out to the S] ($m^3/yrs$)")
    ax1.set_ylabel(r"$Q_s$ [out to the N] ($m^3/yrs$)")
    # ax.plot(out_ds.otime, zumzumN, lw = 1, color = 'k')
    # ax.plot(out_ds.otime, zumzumS, lw = 1, color = 'r')
    plt.tight_layout()
#     plt.savefig("rift_lake_evaporation_%s.png"%(round(tout_ds.Lake__evaporation_rate.values.item(0),2)), dpi = 300)
    plt.savefig("rift_lake_evaporation_explicit.png", dpi = 300)
# plt.savefig()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/var/folders/t0/7zf23f454vn3_kkgrbf4c1500000gp/T/ipykernel_78020/2451138702.py:63: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()


In [19]:
xr.open_zarr("rift_lake_implicit_vs_explicit.zarr/").Lake__method.values

array(['implicit', 'explicit'], dtype='<U8')

In [2]:
from importlib import reload  
import plotter as pol
import warnings;warnings.simplefilter('ignore')
import xarray as xr
import numpy as np
reload(pol)
out_ds = xr.open_zarr("rift_lake_explicit_LakeLuca.zarr/")
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0

pol.anim_lake_cross_section(
	out_ds, # The input ds 
	fname = "rift_lake_explicit_LakeLuca",
	timedim = "otime", # the time dimension
	batch_dim = None,
    batch_val = 3,
	cross_section_dir = 'x', # is the cross section in x or y direction
	xy_cross_section = 10000, # coordinate on the other axis
	color_bedrock = 'gray', # color of the bedrock
	color_sediments = 'orange', # color of the bedrock
	color_water = 'blue', # color of the water
	z_min = 0, # minimum z on the cross_section, if left to none -> min of all
	z_max = 600, # max z on the cross_section, if left to none -> min of all
	# Map parameters
	cmap_elev = 'gist_earth', # cmap of the cross-section
	alpha_hillshade = 0.5, # transparency of the hillshade
    czmin = 0,
    czmax = 600,
    figsize = (12,4.5),
    custom_tickszz = np.arange(0,21000,5000, dtype = np.int),
)

  0%|          | 0/500 [00:00<?, ?it/s]

Movie created at rift_lake_explicit_LakeLuca.mp4
GIF created at rift_lake_explicit_LakeLuca.gif


In [11]:

out_ds.ChonkBase__strati.values[0] = 0
out_ds

<xarray.Dataset>
Dimensions:                                (otime: 500, x: 100, n_labels: 2, y: 100, time: 10000, n_depths_recorded: 50, node: 10000)
Coordinates:
  * n_depths_recorded                      (n_depths_recorded) float64 nan .....
  * node                                   (node) float64 nan 1.0 ... 9.999e+03
  * otime                                  (otime) float64 0.0 ... 9.98e+06
  * time                                   (time) float64 0.0 ... 9.999e+06
  * x                                      (x) float64 0.0 200.0 ... 1.98e+04
  * y                                      (y) float64 0.0 200.0 ... 1.98e+04
Dimensions without coordinates: n_labels
Data variables: (12/36)
    ChonkBase__Q_sout_lab_N                (otime, x, n_labels) float64 dask.array<chunksize=(250, 50, 2), meta=np.ndarray>
    ChonkBase__Q_sout_lab_S                (otime, x, n_labels) float64 dask.array<chunksize=(250, 50, 2), meta=np.ndarray>
    ChonkBase__boundary_conditions         <U11 'periodic_EW'
    ChonkBase__create_folder_strati        bool True
    ChonkBase__depths_res_sed_proportions  int64 10
    ChonkBase__dx                          int64 200
    ...                                     ...
    Uplift__idx_N2                         int64 30
    Uplift__idx_N3                         int64 70
    Uplift__idx_N4                         int64 80
    Uplift__minrate                        int64 0
    Uplift__rate                           float64 0.0005
    Uplift__switch_time                    float64 1e+08

In [ ]:
@nb.njit()
def get_full_sed_volume(vol):
    totvol = 0.
    for i in vol:
        totvol == i
    return totvol
def get_full_sed_volume_one_label(vol,lab,nlab):
    totvol = 0.
    gut = 0
    for i in range(vol.shape[0]):
        if(gut == lab):
            totvol += vol[i]
        gut += 1
        if(gut >= nlab):
            gut = 0
            
    return totvol
        
    

        
        
        

In [14]:
import os


frame = 50
iX_cross_section = 50
nx,ny = out_ds.ChonkBase__nx.item(0), out_ds.ChonkBase__ny.item(0)

tds = out_ds.isel({'otime':frame,'x': iX_cross_section})
pref = os.path.join(tds.ChonkBase__path_strati.item(0),tds.ChonkBase__pref_strati.item(0))
number = str(round(tds.ChonkBase__strati.values.item()))

while(len(number) < 8):
    number = '0' + number
    
    
Zs = np.load(pref+"_zs_" + number + ".npy").reshape(ny,nx,2)
ncells = np.load(pref+"_ncells_" + number + ".npy").reshape(ny,nx)
props = np.load(pref+"_props_" + number + ".npy").reshape(ny,nx)
vol = np.load(pref+"_vol_" + number + ".npy").reshape(ny,nx)

def _anim_lake_cross_section(ds, fig, tt, framedim):
    
    
mov_custom = Movie(ds, _anim_lake_cross_section, input_check = False, framedim = timedim)
mov_custom.width = figsize[0]
mov_custom.height = figsize[1]
mov_custom.save(fname + '.gif', progress=True, overwrite_existing=True, remove_movie=False, gif_resolution_factor=1)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: cannot reshape array of size 69160 into shape (100,100)

In [13]:
Zs = np.load("./strati_test/test1_zs_00000060.npy")

In [14]:
Zs

array([[0.77988553, 0.77988553],
       [0.17343555, 0.17343555],
       [0.6563157 , 0.6563157 ],
       ...,
       [0.19427687, 0.19427687],
       [0.6195631 , 0.6195631 ],
       [0.79809046, 0.79809046]], dtype=float32)

In [17]:
arr = np.random.rand(2,76)

In [18]:
arr.shape

(2, 76)

In [19]:
arr = np.array([[0,0], [1,-1], [2,-2]])

In [31]:
arr.ravel()[np.where(arr.ravel() >= 0 )]

array([0, 0, 1, 2])

In [61]:
arr = np.vstack([np.arange(500), np.arange(500)])
arr.shape

(2, 500)

In [62]:
arr[:, ]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [64]:
np.logical_and(arr[1,:] < 200 , arr[1,:] > 150)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [3]:
m = ipymaps. _map.Map()
m

AttributeError: module 'ipymaps' has no attribute '_map'

In [4]:
help(ipymaps)

Help on package ipymaps:

NAME
    ipymaps

PACKAGE CONTENTS


FILE
    /Users/gailleto/miniforge3/envs/lsdtopytools/lib/python3.8/site-packages/ipymaps/__init__.py


